<a href="https://colab.research.google.com/github/ShaynaBee/Final-Project/blob/main/ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import dependencies
import tensorflow as tf
import numpy as np
import os
import pandas as pd

In [ ]:
# Import files here
from google.colab import files
text = open('high_success.txt', 'rb').read()
text = text.decode(encoding='utf-8')
print ('Total number of characters:', len(text))
print('The first 100 characters:\n', text[:100])

Total number of characters: 8122772
The first 100 characters:
 RT @louietraub: Pfizer tried hiding nine pages of adverse reactions from the public for 75 years unt


In [ ]:
#Get unique caracters in text
vocab = sorted(set(text))

In [ ]:
#Create character to index conversion and back, and assign index values to text     
char_to_idx = {u:i for i, u in enumerate(vocab)}
idx_to_char = np.array(vocab)
text_as_int = np.array([char_to_idx[char] for char in text])

In [ ]:

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) 
# for i in char_dataset.take(5): 
#   print(i.numpy())
seq_length = 100 # The max. length for single input
# examples_per_epoch = len(text)//(seq_length+1) # double-slash for “floor” division
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) 
# for item in sequences.take(5): 
#   print(repr(''.join(idx2char[item.numpy()])))

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:

BUFFER_SIZE = 10000 # TF shuffles the data only within buffers

BATCH_SIZE = 64 # Batch size

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [ ]:

# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:

model = build_model(
    vocab_size = len(vocab), # no. of unique characters
    embedding_dim=embedding_dim, # 256
    rnn_units=rnn_units, # 1024
    batch_size=BATCH_SIZE)  # 64 for the training

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           180992    
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 707)           724675    
                                                                 
Total params: 4,843,971
Trainable params: 4,843,971
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


model.compile(metrics=['accuracy'],optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 30
history = model.fit(dataset, 
                    epochs=EPOCHS, 
                    callbacks=[checkpoint_callback])

Epoch 1/30
1256/1256 [==============================] - 90s 66ms/step - loss: 0.7722 - accuracy: 0.8239
Epoch 2/30
1256/1256 [==============================] - 88s 69ms/step - loss: 0.2837 - accuracy: 0.9346
Epoch 3/30
1256/1256 [==============================] - 88s 69ms/step - loss: 0.2426 - accuracy: 0.9429
Epoch 4/30
1256/1256 [==============================] - 88s 69ms/step - loss: 0.2252 - accuracy: 0.9465
Epoch 5/30
1256/1256 [==============================] - 89s 70ms/step - loss: 0.2169 - accuracy: 0.9482
Epoch 6/30
1256/1256 [==============================] - 88s 69ms/step - loss: 0.2127 - accuracy: 0.9490
Epoch 7/30
 116/1256 [=>............................] - ETA: 1:19 - loss: 0.1254 - accuracy: 0.9702

KeyboardInterrupt: ignored

In [ ]:

tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_6'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            180992    
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 707)            724675    
                                                                 
Total params: 4,843,971
Trainable params: 4,843,971
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, num_generate, temperature, start_string):
  input_eval = [char_to_idx[s] for s in start_string] # string to numbers (vectorizing)
  input_eval = tf.expand_dims(input_eval, 0) # dimension expansion
  text_generated = [] # Empty string to store our results
  model.reset_states() # Clears the hidden states in the RNN

  for i in range(num_generate): #Run a loop for number of characters to generate
    predictions = model(input_eval) # prediction for single character
    predictions = tf.squeeze(predictions, 0) # remove the batch dimension

    # using a categorical distribution to predict the character returned by the model
    # higher temperature increases the probability of selecting a less likely character
    # lower --> more predictable
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # The predicted character as the next input to the model
    # along with the previous hidden state
    # So the model makes the next prediction based on the previous character
    input_eval = tf.expand_dims([predicted_id], 0) 
    # Also devectorize the number and add to the generated text
    text_generated.append(idx_to_char[predicted_id]) 

  return (start_string + ''.join(text_generated))

In [ ]:

generated_text = generate_text(
                    model, 
                    num_generate=260, 
                    temperature=1, 
                    start_string=u"Data")
print(generated_text)

DataYᱬ_KFTLEçNNwI: Ishappensires until after the mid-terms or until this deal is fully c…
RT @donwinslow: If you are considering leaving Twitter, I urge you to wait AT LEAST until after the mid-terms or until this deal is fully c…
RT @buitengebieden_: Dolick wha
